# Can you SEE MY SCREEN?

<span style="color: red;"> 
How do we connect LLMs with Tools?
</span>


In [2]:
from openai import OpenAI
import os

client = OpenAI()

# if you don't have an API key as an environment variable you can set it here

# os.environ["OPENAI_API_KEY"] = ""

In [3]:
def llm_response(prompt: str):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

llm_response("How do we connect LLMs with Tools?")

"Connecting large language models (LLMs) with tools involves integrating external functionality so that the model can effectively perform specific tasks beyond its built-in capabilities. Here’s how you can connect LLMs with tools:\n\n### 1. **API Integration**\n   - **API Setup:** Tools can provide APIs (Application Programming Interfaces) that LLMs can call to perform functions, such as retrieving data, sending notifications, or executing commands.\n   - **Calling APIs:** When the model identifies a need that requires external data or functionality, it can format a request to the tool's API and process the response.\n   - **Example:** An LLM could call a weather API to provide real-time weather updates.\n\n### 2. **Pre-defined Functions**\n   - **Function Wrapping:** Create predefined functions that encapsulate tool functionality. The LLM can then call these functions directly.\n   - **Function Selection:** The LLM can use context from a conversation to decide which function to call a

We could ask the model to:

1. Generate Python code and then give the model the ability to run that code somewhere
2. Create the Python function that executes the task, then somehow give the model the ability to run that function autonomously

Option number 2 is what we call FUNCTION CALLING!

# Function Calling from Scratch

1. Write a Python function
2. Make that function "available" to the LLM model of choice (in this case will be gpt-4o/gpt-4o-mini)
3. Test if the model can execute that function properly
   1. Prepare the payload for the function (the arguments for it...)
   2. Write the function call
   3. We run the function call ourselves
   4. We inspect the results

In [4]:
def create_folder(folder_path: str):
    os.makedirs(folder_path, exist_ok=True)
    return f"Folder created at {folder_path}"

create_folder("./pancakes-are-the-best")

'Folder created at ./pancakes-are-the-best'

In [7]:
!ls -d ./* | grep pancakes

./pancakes-are-the-best


In [8]:
def write_file(file_path: str, content: str):
    with open(file_path, "w") as file:
        file.write(content)
    return f"File created at {file_path}"

write_file("./pancakes-are-the-best/pancakes-are-the-best.txt", "Pancakes are the best!")

'File created at ./pancakes-are-the-best/pancakes-are-the-best.txt'

In [9]:
!cat pancakes-are-the-best/pancakes-are-the-best.txt

Pancakes are the best!

# LLM + Python Functions

The most silly way possible first!

In [11]:
prompt_with_function_information = """
You are a personal assistant with desktop capabilities. 
Users will ask you to perform tasks and you will execute those tasks by calling one or more of the following functions:

- create_folder(folder_path: str)
- write_file(file_path: str, content: str)

For example if the user asks: 

'Create a folder called lucas-teaches-function-callling'

Your output should be code like this:
'create_folder("./lucas-teaches-function-callling")'

'Create a folder called pancakes-are-the-best and write a file called pancakes-are-the-best.txt with the content "Pancakes are the best!"'

Your output should be code like this:

['create_folder("./pancakes-are-the-best")', 'write_file("./pancakes-are-the-best/pancakes-are-the-best.txt", "Pancakes are the best!")']

Your OUTPUT should ALWAYS BE python code only.

Here is the user prompt:
"""


user_prompt = "Create a file named pancakes-are-better-than-waffles.md with a one paragraph summary for why pancakes are better, \
make sure to address RS a student from my course who dared to say avocado waffles are better."

prompt = prompt_with_function_information + user_prompt

llm_response(prompt)

'```python\nwrite_file("./pancakes-are-better-than-waffles.md", "Pancakes are undoubtedly superior to avocado waffles, RS! Their fluffy texture and delightful flavors can never be matched. While avocado waffles may have their charm, pancakes offer endless possibilities for toppings, from syrup to fruits, making each bite a delicious experience. Plus, the comforting aroma of pancakes cooking on a Sunday morning is simply unrivaled. So, next time you reach for breakfast, remember that pancakes reign supreme!")\n```'

In [12]:
def clean_python_output(prompt: str):
    return prompt.replace("```python\n", "").replace("```", "")

clean_python_output(llm_response(prompt))

'write_file("./pancakes-are-better-than-waffles.md", "Pancakes are better than waffles because they offer a fluffy texture that absorbs syrup and toppings beautifully, creating a delightful flavor experience in every bite. Unlike avocado waffles, which can be overly savory and may not satisfy a sweet tooth, pancakes provide a comforting sweetness that pairs perfectly with fruits, whipped cream, or even chocolate chips. RS, while we appreciate your love for avocado waffles, nothing quite matches the enjoyment of a stack of warm, buttery pancakes!")\n'

In [13]:
output = clean_python_output(llm_response(prompt))

output

'write_file("./pancakes-are-better-than-waffles.md", "Pancakes are undeniably better than waffles, RS! While some may argue that avocado waffles have a unique flavor, pancakes offer a fluffy, comforting texture that perfectly complements a variety of toppings, from fresh fruits to maple syrup. Their adaptability and classic appeal make them a quintessential breakfast choice that can satisfy any palate.")\n'

Let's find a way to execute this code!

In [14]:
exec(output)

We connected LLM + FUNCTION via some simple PROMPT magic!

Pancakes are undeniably better than waffles, RS! While some may argue that avocado waffles have a unique flavor, pancakes offer a fluffy, comforting texture that perfectly complements a variety of toppings, from fresh fruits to maple syrup. Their adaptability and classic appeal make them a quintessential breakfast choice that can satisfy any palate.